In [ ]:
import os
import sys
# Make sure all code is in the PATH.
sys.path.append(
    os.path.normpath(
        os.path.join(
            os.environ["HOME"], "Projects", "cosine_neutral_loss", "src"
        )
    )
)

In [ ]:
import functools
import lzma
import re

import Levenshtein
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numba as nb
import numpy as np
import pandas as pd
import pyteomics.mgf
import seaborn as sns
import spectrum_utils.spectrum as sus
import tqdm.notebook as tqdm
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec

import similarity

In [ ]:
# Plot styling.
plt.style.use(["seaborn-white", "seaborn-paper"])
plt.rc("font", family="sans-serif")
sns.set_palette(["#9e0059", "#6da7de", "#dee000"])
sns.set_context("paper", font_scale=1.)

## Analysis settings

In [ ]:
# Spectra and spectrum pairs to include with the following settings.
charges = 1,
min_n_peaks = 6
fragment_mz_tolerance = 0.1
min_mass_diff = 1    # Da
max_mass_diff = 200    # Da
# If defined, only consider a specific delta m/z between spectrum pairs.
# selected_mass_diff = 15.9949
selected_mass_diff = None
mass_diff_tol = 0.1

In [ ]:
# Profile spectra contain 0 intensity values.
@nb.njit
def is_centroid(intensity_array):
    return np.all(intensity_array > 0)


# Assumes that the spectra are sorted by ascending precusor m/z.
@nb.njit
def generate_pairs(
    spectrum_indexes,
    masses,
    min_mass_diff,
    max_mass_diff,
    selected_mass_diff,
    mass_diff_tol,
):
    for i in range(len(spectrum_indexes)):
        j = i + 1
        while (
            j < len(spectrum_indexes) and
            min_mass_diff < masses[j] - masses[i] < max_mass_diff
        ):
            mass_diff = masses[j] - masses[i]
            if (
                selected_mass_diff is None or
                abs(selected_mass_diff - mass_diff) < mass_diff_tol
            ):
                yield spectrum_indexes[i]
                yield spectrum_indexes[j]
            j += 1

## Data IO

In [ ]:
# Read all spectra from the MGF.
# ALL_GNPS_NO_PROPOGATED (retrieved on 2022-05-12) downloaded from
# https://gnps-external.ucsd.edu/gnpslibrary

# Spectrum quality filtering:
#   - Don't include propagated spectra (LIBRARYQUALITY==4).
#   - Don't include multiply charged molecules.
#   - Don't include spectra with invalid precursor m/z (0).
#   - Don't include spectra with too few peaks (minimum 6).
#   - Only include positive ion mode spectra.
#   - Only include spectra with [M+H]+ adducts.
#   - Only include centroid data (does not contain zero intensity values).
spectra = []
filename = ("../data/external/ALL_GNPS_NO_PROPOGATED.mgf.xz")
with lzma.open(filename, "rt") as xz_in:
    with pyteomics.mgf.MGF(xz_in) as f_in:
        for spectrum_dict in tqdm.tqdm(f_in):
            if (
                int(spectrum_dict["params"]["libraryquality"]) <= 3 and
                int(spectrum_dict["params"]["charge"][0]) in charges and
                float(spectrum_dict["params"]["pepmass"][0]) > 0 and
                len(spectrum_dict["m/z array"]) >= min_n_peaks and
                spectrum_dict["params"]["ionmode"] == "Positive" and
                spectrum_dict["params"]["name"].rstrip().endswith(" M+H") and
                is_centroid(spectrum_dict["intensity array"])
            ):
                spec = sus.MsmsSpectrum(
                    spectrum_dict["params"]["spectrumid"],
                    float(spectrum_dict["params"]["pepmass"][0]),
                    int(spectrum_dict["params"]["charge"][0]),
                    spectrum_dict["m/z array"],
                    spectrum_dict["intensity array"],
                )
                spec.library = spectrum_dict["params"]["organism"]
                spec.smiles = spectrum_dict["params"]["smiles"]
                spec.remove_precursor_peak(0.1, "Da")
                spectra.append(spec)

In [ ]:
# Extract the metadata (library identifier and precursor charge and m/z).
identifiers, libraries, smiles, charges, mzs = [], [], [], [], []
metadata = []
for spectrum in spectra:
    identifiers.append(spectrum.identifier)
    libraries.append(spectrum.library)
    smiles.append(spectrum.smiles)
    charges.append(spectrum.precursor_charge)
    mzs.append(spectrum.precursor_mz)
metadata = pd.DataFrame(
    {
        "id": identifiers,
        "library": libraries,
        "smiles": smiles,
        "charge": charges,
        "mz": mzs,
    }
)

## Compute spectrum-spectrum similarities

In [ ]:
# Extract indexes for the relevant pairs of spectra.
pairs = []
for charge in np.arange(
        metadata["charge"].min(),
        metadata["charge"].max() + 1,
    ):
    # Make sure the spectra are sorted by ascending precursor m/z.
    metadata_charge = (metadata[metadata["charge"] == charge]
                       .copy()
                       .sort_values("mz")
                       .reset_index())
    # Generate pairs with the specified precursor m/z difference.
    pairs.append(
        np.fromiter(
            generate_pairs(
                metadata_charge["index"].values,
                metadata_charge["mz"].values,
                min_mass_diff,
                max_mass_diff,
                selected_mass_diff,
                mass_diff_tol,
            ),
            np.int32)
        .reshape((-1, 2))
    )
pairs = np.vstack(pairs)

In [ ]:
# Compute similarities between spectrum pairs.
cosines, modified_cosines, neutral_losses = [], [], []
similarities = []
for i, j in tqdm.tqdm(pairs):
    cos = similarity.cosine(spectra[i], spectra[j], fragment_mz_tolerance)
    mod_cos = similarity.modified_cosine(
        spectra[i], spectra[j], fragment_mz_tolerance
    )
    nl = similarity.neutral_loss(
        spectra[i], spectra[j], fragment_mz_tolerance
    )
    similarities.append(
        (cos[0], cos[1], mod_cos[0], mod_cos[1], nl[0], nl[1])
    )
similarities = pd.DataFrame(
    similarities,
    columns=[
        "cosine",
        "cosine_explained",
        "modified_cosine",
        "modified_cosine_explained",
        "neutral_loss",
        "neutral_loss_explained",
    ]
)
similarities[["pair1", "pair2"]] = pairs
similarities["id1"] = metadata.loc[pairs[:, 0], "id"].values
similarities["id2"] = metadata.loc[pairs[:, 1], "id"].values
similarities["smiles1"] = metadata.loc[pairs[:, 0], "smiles"].values
similarities["smiles2"] = metadata.loc[pairs[:, 1], "smiles"].values
similarities["charge1"] = metadata.loc[pairs[:, 0], "charge"].values
similarities["charge2"] = metadata.loc[pairs[:, 1], "charge"].values
similarities["mz1"] = metadata.loc[pairs[:, 0], "mz"].values
similarities["mz2"] = metadata.loc[pairs[:, 1], "mz"].values
similarities.to_parquet("gnps_libraries.parquet")

## Results plotting

In [ ]:
print(f"Number of spectrum pairs: {len(similarities):,}")
print(
    f"Spectrum pairs where neutral loss outperforms cosine: "
    f"{(similarities['neutral_loss'].round(5) > similarities['cosine'].round(5)).sum() / len(similarities):.1%}"
)
print(
    f"Spectrum pairs where neutral loss outperforms modified cosine: "
    f"{(similarities['neutral_loss'].round(5) > similarities['modified_cosine'].round(5)).sum() / len(similarities):.1%}"
)

In [ ]:
mosaic = """
11111.
222223
222223
222223
222223
222223
"""

bins = 100
tick_locators = mticker.FixedLocator(np.arange(0, bins + 1, bins / 4))
tick_labels = np.asarray([f"{a:.2f}" for a in np.arange(0, 1.01, 0.25)])

with sns.plotting_context("paper", font_scale=1.6):
    fig = plt.figure(constrained_layout=True, figsize=(7.2 * 2, 7.2 / 1.618 * 2))
    gs = GridSpec(2, 3, figure=fig)
    axes_labeled = []
    
    # Top panel: Compare different similarities.
    axes_left = fig.add_subfigure(gs[0, 0]).subplot_mosaic(mosaic)
    axes_middle = fig.add_subfigure(gs[0, 1]).subplot_mosaic(mosaic)
    axes_right = fig.add_subfigure(gs[0, 2]).subplot_mosaic(mosaic)
    cbar_ax = fig.add_axes([-0.04, 0.65, 0.02, 0.15])
    axes_labeled.append(cbar_ax)
    
    labels = np.asarray([
        ["cosine", "modified_cosine"],
        ["neutral_loss", "cosine"],
        ["neutral_loss", "modified_cosine"]
    ])

    for i, (axes, (xlabel, ylabel)) in enumerate(
        zip([axes_left, axes_middle, axes_right], labels)
    ):
        # Plot heatmaps.
        hist, _, _ = np.histogram2d(
            similarities[xlabel],
            similarities[ylabel],
            bins=bins,
            range=[[0, 1], [0, 1]],
        )
        hist /= len(similarities)
        heatmap = sns.heatmap(
            np.rot90(hist),
            vmin=0.0,
            vmax=0.001,
            cmap="viridis",
            cbar=i == 2,
            cbar_kws={"format": mticker.StrMethodFormatter("{x:.3%}")},
            cbar_ax=cbar_ax if i == 2 else None,
            square=True,
            xticklabels=False,
            yticklabels=False,
            ax=axes["2"],
            norm=LogNorm(vmax=0.001),
        )
        axes["2"].yaxis.set_major_locator(tick_locators)
        axes["2"].set_yticklabels(tick_labels[::-1])
        axes["2"].xaxis.set_major_locator(tick_locators)
        axes["2"].set_xticklabels(tick_labels)
        for _, spine in heatmap.spines.items():
            spine.set_visible(True)
        axes["2"].set_xlabel(xlabel.replace("_", " ").capitalize())
        axes["2"].set_ylabel(ylabel.replace("_", " ").capitalize())

        axes["2"].plot(
            [0, bins], [bins, 0], color="black", linestyle="dashed"
        )

        sns.despine(ax=axes["2"])

        # Plot density plots.
        sns.kdeplot(
            data=similarities,
            x=xlabel,
            clip=(0, 1),
            legend=True,
            color="black",
            fill=True,
            ax=axes["1"],
        )
        axes["1"].set_xlim(0, 1)
        axes["1"].xaxis.set_ticklabels([])
        axes["1"].yaxis.set_major_locator(tick_locators)
        axes["1"].set_yticks([])
        sns.despine(ax=axes["1"], left=True)
        sns.kdeplot(
            data=similarities,
            y=ylabel,
            clip=(0, 1),
            legend=True,
            color="black",
            fill=True,
            ax=axes["3"],
        )
        axes["3"].set_ylim(0, 1)
        axes["3"].yaxis.set_ticklabels([])
        axes["3"].xaxis.set_major_locator(tick_locators)
        axes["3"].set_xticks([])
        sns.despine(ax=axes["3"], bottom=True)
        for ax in [axes[c] for c in "13"]:
            ax.set_xlabel("")
            ax.set_ylabel("")
            
    cbar_ax.set_ylabel("Proportion of pairs")
    cbar_ax.yaxis.set_label_position("left")
    cbar_ax.spines["outline"].set(visible=True, lw=.8, edgecolor="black")
    
    # Bottom panel: Compare similarities vs explained intensity.
    axes_left = fig.add_subfigure(gs[1, 0]).subplot_mosaic(mosaic)
    axes_middle = fig.add_subfigure(gs[1, 1]).subplot_mosaic(mosaic)
    axes_right = fig.add_subfigure(gs[1, 2]).subplot_mosaic(mosaic)
    cbar_ax = fig.add_axes([-0.04, 0.15, 0.02, 0.15])
    axes_labeled.append(cbar_ax)
    
    labels = np.asarray([
        ["cosine_explained", "cosine"],
        ["neutral_loss_explained", "neutral_loss"],
        ["modified_cosine_explained", "modified_cosine"],
    ])

    for i, (axes, (xlabel, ylabel)) in enumerate(
        zip([axes_left, axes_middle, axes_right], labels)
    ):
        # Plot heatmaps.
        hist, _, _ = np.histogram2d(
            similarities[xlabel],
            similarities[ylabel],
            bins=bins,
            range=[[0, 1], [0, 1]],
        )
        hist /= len(similarities)
        heatmap = sns.heatmap(
            np.rot90(hist),
            vmin=0.0,
            vmax=0.001,
            cmap="viridis",
            cbar=i == 2,
            cbar_kws={"format": mticker.StrMethodFormatter("{x:.3%}")},
            cbar_ax=cbar_ax if i == 2 else None,
            square=True,
            xticklabels=False,
            yticklabels=False,
            ax=axes["2"],
            norm=LogNorm(vmax=0.001),
        )
        axes["2"].yaxis.set_major_locator(tick_locators)
        axes["2"].set_yticklabels(tick_labels[::-1])
        axes["2"].xaxis.set_major_locator(tick_locators)
        axes["2"].set_xticklabels(tick_labels)
        axes["2"].xaxis.set_major_formatter(mticker.PercentFormatter())
        for _, spine in heatmap.spines.items():
            spine.set_visible(True)
        axes["2"].set_xlabel("Explained intensity")
        axes["2"].set_ylabel(ylabel.replace("_", " ").capitalize())

        sns.despine(ax=axes["2"])

        # Plot density plots.
        sns.kdeplot(
            data=similarities,
            x=xlabel,
            clip=(0, 1),
            legend=True,
            color="black",
            fill=True,
            ax=axes["1"],
        )
        axes["1"].set_xlim(0, 1)
        axes["1"].xaxis.set_ticklabels([])
        axes["1"].yaxis.set_major_locator(tick_locators)
        axes["1"].set_yticks([])
        sns.despine(ax=axes["1"], left=True)
        sns.kdeplot(
            data=similarities,
            y=ylabel,
            clip=(0, 1),
            legend=True,
            color="black",
            fill=True,
            ax=axes["3"],
        )
        axes["3"].set_ylim(0, 1)
        axes["3"].yaxis.set_ticklabels([])
        axes["3"].xaxis.set_major_locator(tick_locators)
        axes["3"].set_xticks([])
        sns.despine(ax=axes["3"], bottom=True)
        for ax in [axes[c] for c in "13"]:
            ax.set_xlabel("")
            ax.set_ylabel("")
            
    cbar_ax.set_ylabel("Proportion of pairs")
    cbar_ax.yaxis.set_label_position("left")
    cbar_ax.spines["outline"].set(visible=True, lw=.8, edgecolor="black")
    
    # Subplot labels.
    for y, label in zip([1, 0.5], "ab"):
        fig.text(
            -0.05, y, label, fontdict=dict(fontsize="xx-large", weight="bold")
        )

    # Save figure.
    plt.savefig("gnps_libraries.png", dpi=300, bbox_inches="tight")
    plt.show()
    plt.close()